<a href="https://colab.research.google.com/github/tempiatine/PySpark-tasks/blob/main/Movies_actors_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=c7aaa02fe3500ba16fec71d1efbfdb1643eb1c12fdd73570e058557b51b1b599
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [17]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV actors & movies").getOrCreate()
from pyspark.sql.functions import col, count, countDistinct, max

# Чтение CSV-файлов
movies_df = spark.read.csv("/content/sample_data/movies.csv", header=True, inferSchema=True)
actors_df = spark.read.csv("/content/sample_data/actors.csv", header=True, inferSchema=True)
movies_actors_df = spark.read.csv("/content/sample_data/movie_actors.csv", header=True, inferSchema=True)

# Регистрация DataFrame как временные таблицы
movies_df.createOrReplaceTempView("movies")
actors_df.createOrReplaceTempView("actors")
movies_actors_df.createOrReplaceTempView("movie_actors")

# Выполнение SQL-запроса для нахождения топ-5 жанров по количеству фильмов
genre_sum = spark.sql("""
SELECT genre, count(movie_id) as num_movies
FROM movies
GROUP BY genre
ORDER BY count(movie_id) DESC
LIMIT 5
""")

# Выполнение SQL-запроса для нахождения актера с наибольшим количеством фильмов
max_movies_actor = spark.sql("""
SELECT a.name, count(DISTINCT ma.movie_id) as num_movies
FROM movie_actors ma
JOIN actors a
ON ma.actor_id = a.actor_id
GROUP BY a.name
ORDER BY count(DISTINCT ma.movie_id) DESC
LIMIT 1
""")

genre_avg_budget = spark.sql("""
SELECT genre, avg(budget) as avg_budget
FROM movies
GROUP BY genre
ORDER BY avg(budget) DESC
""")

pop_country = spark.sql("""
SELECT m.title, a.country, count(DISTINCT ma.actor_id) as num_actors
FROM movie_actors ma
JOIN actors a
ON ma.actor_id = a.actor_id
JOIN movies m
ON ma.movie_id = m.movie_id
GROUP BY a.country, m.title
HAVING num_actors > 1
""")

genre_sum.show()
max_movies_actor.show()
genre_avg_budget.show()
pop_country.show()


+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_24|         5|
+--------+----------+

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
|Horror|      8.7281876775E7|
|Sci-Fi|       7.809715175E7|
| Drama| 6.076021856166667E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
+------+--------------------+

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
|Movie_15|    India|         2|
| Movie_7|      USA|         2|
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
| Movie_1|    India|         2|
|Movie_10|       UK|         2|
|Movie_18|Australia|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+

